<a href="https://colab.research.google.com/github/shahd1995913/Machine-Learning/blob/main/6.%20Recommendation%20System%20/Anime_Recommendation_System_by_TAH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [youtub](https://www.youtube.com/watch?v=kccT0FVK6OY&list=PLZoTAELRMXVN7QGpcuN-Vg35Hgjp3htvi&index=3)
## [Github](https://github.com/krishnaik06/Recommendation_complete_tutorial/blob/master/KNN%20Movie%20Recommendation/KNNRecommendation.ipynb)
## [Kaggel](https://www.kaggle.com/competitions/anime-recommendation-system-by-tah/data?select=train.csv)

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download anime-recommendation-system-by-tah
! unzip /content/anime-recommendation-system-by-tah.zip
import pandas as pd

In [ ]:
import pandas as pd
rating_complete = pd.read_csv("/content/rating_complete.csv")
anime = pd.read_csv("/content/anime.csv")
movies_df = pd.read_csv('/content/anime.csv',usecols=['MAL_ID','Name'],dtype={'MAL_ID': 'int32', 'Name': 'str'})
rating_df=pd.read_csv('/content/rating_complete.csv',usecols=['user_id', 'anime_id', 'rating'],
    dtype={'user_id': 'int32', 'anime_id': 'int32', 'rating': 'float32'})
movies_df.rename(columns = {'MAL_ID':'anime_id'}, inplace = True)
df = pd.merge(rating_df,movies_df,on='anime_id')

In [ ]:
combine_movie_rating = df.dropna(axis = 0, subset = ['Name'])
movie_ratingCount = (combine_movie_rating.
     groupby(by = ['Name'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['Name', 'totalRatingCount']]
    )
movie_ratingCount.head()

In [ ]:
rating_with_totalRatingCount = combine_movie_rating.merge(movie_ratingCount, left_on = 'Name', right_on = 'Name', how = 'left')
rating_with_totalRatingCount.head()

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(movie_ratingCount['totalRatingCount'].describe())

In [ ]:
popularity_threshold = 50
rating_popular_movie= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_movie.head()

In [ ]:
rating_popular_movie.shape

In [ ]:
## First lets create a Pivot matrix
movie_features_df=rating_popular_movie.pivot_table(index='Name',columns='anime_id',values='rating').fillna(0)
# movie_features_df=rating_popular_movie.pivot_table(index='Name',columns='anime_id',values='rating').dropna()

movie_features_df.head()

In [ ]:
from scipy.sparse import csr_matrix
movie_features_df_matrix = csr_matrix(movie_features_df.values)
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

In [ ]:
movie_features_df.shape

In [ ]:
query_index = np.random.choice(movie_features_df.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

In [ ]:
movie_features_df.head()


In [ ]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_features_df.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))